In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard - Michael Garcia'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
             style={'width': '300px'}),
    html.Hr(),

    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset (All)', 'value': 'RESET'},
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DISASTER'}
            ],
            value='RESET',
            inline=True
        )
    ]),
    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[0]
    ),

    html.Br(),
    html.Hr(),

    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):

    # 🔹 Filter queries updated to match actual MongoDB field names
    if filter_type == "WATER":
        query = {
            "animal_type": "Dog",
            "age_upon_outcome_in_weeks": {"$lte": 104},  # 2 years
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever Mix", 
                              "Newfoundland Mix", "Labrador Retriever/Australian Cattle Dog"]}
        }

    elif filter_type == "MOUNTAIN":
        query = {
            "animal_type": "Dog",
            "age_upon_outcome_in_weeks": {"$lte": 104},
            "breed": {"$in": ["German Shepherd Mix", "Alaskan Malamute Mix", "Old English Sheepdog Mix"]}
        }

    elif filter_type == "DISASTER":
        query = {
            "animal_type": "Dog",
            "age_upon_outcome_in_weeks": {"$lte": 104},
            "breed": {"$in": ["Doberman Pinscher Mix", "German Shepherd Mix", 
                              "Golden Retriever Mix", "Bloodhound Mix"]}
        }

    else:
        query = {}

    data = pd.DataFrame.from_records(db.read(query))
    if '_id' in data:
        data.drop(columns=['_id'], inplace=True)

    return data.to_dict('records')


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return

    dff = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Breed Distribution')
        )
    ]


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None:
        return
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    dff = pd.DataFrame.from_dict(viewData)

    # 🔹 Updated to use actual MongoDB latitude/longitude fields
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                    dl.Tooltip(dff.iloc[row]['name']),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row]['name'])
                    ])
                ])
            ])
    ]


# Run app and display result in jupyterlab mode
app.run_server(port=8050, mode="inline")

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: 'NoneType' object is not iterable

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: 'NoneType' object is not iterable

